Requirements

In [1]:
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install pycocotools
!pip install cpython
!pip install wget
!git clone https://github.com/pytorch/vision

  Created wheel for cpython: filename=cPython-0.0.6-py3-none-any.whl size=4913 sha256=4bd9a415392710ddf76a47b4c513e4fc6219a7b572412f4ffe29a9aa384697b6
  Stored in directory: /root/.cache/pip/wheels/88/92/ea/c32ad929e979a7303e010b29c736c793368f6f61c8c9902865
Successfully built cpython
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=1dd4ba7a5bc741f5c234fce677197d2939c720b82cfd18bd09dfc7a9ec253c8c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Cloning into 'vision'...
remote: Enumerating objects: 129988, done.
remote: Total 129988 (delta 0), reused 0 (delta 0), pack-reused 129988
Receiving objects: 100% (129988/129988), 252.30 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (113966/113966), done.


In [2]:
!git clone https://github.com/st0bb3n/ObjectDetection-Drinks

Cloning into 'ObjectDetection-Drinks'...
remote: Enumerating objects: 1252, done.
remote: Counting objects: 100% (1252/1252), done.
remote: Compressing objects: 100% (1144/1144), done.
remote: Total 1252 (delta 107), reused 1243 (delta 102), pack-reused 0
Receiving objects: 100% (1252/1252), 26.56 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (107/107), done.


Set Dataset pathing and torchvision reference modules

In [3]:
import sys
sys.path.insert(0, "/content/vision/references/detection")

dataset_path = "/content/drinkscoco"

Download Dataset

In [5]:
import wget
import shutil

url = "https://github.com/st0bb3n/ObjectDetection-Drinks/releases/download/Dataset/drinkscoco.zip"
x = wget.download(url, "data.zip")

shutil.unpack_archive("data.zip","drinkscoco")

url = "https://github.com/st0bb3n/ObjectDetection-Drinks/releases/download/Dataset/trainedmodel.pth"
x = wget.download(url, "trainedmodel.pth")

Main function + Dataset Class. To be run first.

In [6]:
import numpy as np 
import pandas as pd 
import os
import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
from PIL import Image
import cv2
import albumentations as A  # our data augmentation library
import matplotlib.pyplot as plt
import datetime
import time
from torchvision.utils import draw_bounding_boxes
from pycocotools.coco import COCO
from engine import evaluate, train_one_epoch
import utils

class Drinks(datasets.VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None, transforms=None):
        # the 3 transform parameters are required for datasets.VisionDataset
        super().__init__(root, transforms, transform, target_transform)
        self.split = split #train, valid, test
        self.coco = COCO(os.path.join(root, split, "_annotations.coco.json")) # annotations stored here
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.ids = [id for id in self.ids if (len(self._load_target(id)) > 0)]
    
    def _load_image(self, id: int):
        path = self.coco.loadImgs(id)[0]['file_name']
        image = cv2.imread(os.path.join(self.root, self.split, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

        return image/255

    def _load_target(self, id):
        return self.coco.loadAnns(self.coco.getAnnIds(id))
    
    def __getitem__(self, index):
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)
        target = copy.deepcopy(self._load_target(id))
        
        boxes = [t['bbox'] + [t['category_id']] for t in target] 
        
        new_boxes = [] # convert from xywh to xyxy
        for box in boxes:
            xmin = box[0]
            xmax = xmin + box[2]
            ymin = box[1]
            ymax = ymin + box[3]
            new_boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.tensor(new_boxes, dtype=torch.float32)
        
        targ = {} # here is our transformed target
        targ['boxes'] = boxes
        targ['labels'] = torch.tensor([t['category_id'] for t in target], dtype=torch.int64)
        #targ['image_id'] = torch.tensor([t['image_id'] for t in target])
        targ['image_id'] = torch.tensor([index])
        targ['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) # we have a different area
        targ['iscrowd'] = torch.tensor([t['iscrowd'] for t in target], dtype=torch.int64)
        
        return torchvision.transforms.ToTensor()(image), targ # scale images

    def __len__(self):
        return len(self.ids)

test.py

In [ ]:
coco = COCO(os.path.join(dataset_path, "train", "_annotations.coco.json"))
categories = coco.cats
n_classes = len(categories.keys())
classes = [i[1]['name'] for i in categories.items()]

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)

model.load_state_dict(torch.load("/content/trainedmodel.pth"))

device = torch.device("cuda")
model = model.to(device)

train_dataset = Drinks(root=dataset_path, transforms=torchvision.transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=utils.collate_fn)

#images,targets = next(iter(train_loader))
#images = list(image for image in images)
#targets = [{k:v for k, v in t.items()} for t in targets]
#output = model(images, targets)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

dataset = Drinks(root=dataset_path, transforms=torchvision.transforms.ToTensor())
test_dataset = Drinks(root=dataset_path, split="test", transforms=torchvision.transforms.ToTensor())

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=2,
        collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn)

evaluate(model, data_loader_test, device=device)

train.py

In [6]:
coco = COCO(os.path.join(dataset_path, "train", "_annotations.coco.json"))
categories = coco.cats
n_classes = len(categories.keys())
classes = [i[1]['name'] for i in categories.items()]

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features # we need to change the head
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)

train_dataset = Drinks(root=dataset_path, transforms=torchvision.transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=utils.collate_fn)

#images,targets = next(iter(train_loader))
#images = list(image for image in images)
#targets = [{k:v for k, v in t.items()} for t in targets]
#output = model(images, targets)
#output = model(images, targets)

device = torch.device("cuda") # use GPU to train
model = model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

dataset = Drinks(root=dataset_path, transforms=torchvision.transforms.ToTensor())
test_dataset = Drinks(root=dataset_path, split="test", transforms=torchvision.transforms.ToTensor())

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=2,
        collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn)

num_epochs=10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=50)
    #lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)
    
model.eval()
torch.save(model.state_dict(), "trainedmodel.pth")
torch.cuda.empty_cache()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Epoch: [0]  [  0/236]  eta: 0:02:17  lr: 0.000053  loss: 1.7081 (1.7081)  loss_classifier: 1.3071 (1.3071)  loss_box_reg: 0.3361 (0.3361)  loss_objectness: 0.0605 (0.0605)  loss_rpn_box_reg: 0.0044 (0.0044)  time: 0.5846  data: 0.2180  max mem: 543
Epoch: [0]  [ 50/236]  eta: 0:00:41  lr: 0.002178  loss: 0.8718 (1.0476)  loss_classifier: 0.3811 (0.5742)  loss_box_reg: 0.4500 (0.4257)  loss_objectness: 0.0265 (0.0420)  loss_rpn_box_reg: 0.0042 (0.0057)  time: 0.1971  data: 0.0243  max mem: 690
Epoch: [0]  [100/236]  eta: 0:00:27  lr: 0.004304  loss: 0.9343 (1.0872)  loss_classifier: 0.3211 (0.5080)  loss_box_reg: 0.5989 (0.5445)  loss_objectness: 0.0121 (0.0290